## Номер документа

Номер документа часто указывается после символа No или N и может состоять из цифр, букв и спецсимволов. Примеры номеров:

* 67-ФЗ,

* 324-01-ЗМО,

* 824н.

Номер документа необходимо извлечь из текста «как есть», не подвергая дополнительной обработке.
Поле number должно содержать извлеченную строку. Чтобы скомпенсировать влияние ошибок распознавания символов в исходных текстах, при сравнении предсказанного и настоящего номера проверяющей системой регистр букв не учитывается.


Для каждого типа документа свой извлекатор:

* указ - в конце документа на отдельной строке № 807, может быть и в начале № 256-уг

* постановление, распоряжение, приказ - в начале после даты и места (или перед местом) в конце строки № 688-п, №_70-пп_, 63-пр,  № 25/2-г

* закон, федеральный закон - в конце документа № 89-ОЗ

Особые случаи:

* № _ 643-уГ

* 20 ноября 2014 года

№ 596-5-ЗКО

в постановлении в конце документа

* 02 ноября 2015 г. л 23/557-П в постановлении в начале (будем выбирать в качестве номера окончание строки)

In [1]:
import re
import json
import numpy as np

In [74]:
month2number = {
    'января': '01',
    'февраля': '02',
    'марта': '03',
    'апреля': '04',
    'мая': '05',
    'июня': '06',
    'июля': '07',
    'августа': '08',
    'сентября': '09',
    'октября': '10',
    'ноября': '11',
    'декабря': '12'
}

NUM_DATE_EXPR_1 = re.compile(r'(\d?\d) ([а-я]+) (\d\d\d\d)[^№\n]*\n*[^№\n]*\n*№[ _]*([\w\-а-я/]+)')
NUM_DATE_EXPR_2 = re.compile(r'(\d\d\.\d\d\.\d\d\d\d)[^№\n]*\n*[^№\n]*\n*№[ _]*([\w\-а-я/]+)')

def process_match(match):
    day = match[0] if len(match[0]) == 2 else '0' + match[0]
    month = month2number[match[1]] if match[1] in month2number else match[1]
    year = match[2]
    return f'{day}.{month}.{year}'

In [123]:
def extract_number(doc, doc_type, doc_date):
    doc = doc.lower()
    number = ""
    
    if doc_type == 'постановление':
        match = re.search(r'постановление\n([^\n\d]*\n)?([^\n\d]*\n)?([^\n\d]*\n)?'
                          '(от[_\- ]+?)?\d?\d [а-я]+ \d\d\d\d[^\n]*[ _]\$?&?([\w\-/а-я]+)\n', doc)
        if match:
            return re.sub('ппп', 'пп', re.sub('_', '', match.group(5)))
        match = re.search(r'постановление\n([^\n\d]*\n)?([^\n\d]*\n)?([^\n\d]*\n)?'
                          '[а-я_\- ]*\d\d\.\d\d\.\d\d\d\d[^\n]*[ _]\$?&?([\w\-/а-я]+)\n', doc)
        if match:
            return re.sub('ппп', 'пп', re.sub('_', '', match.group(4)))
        
    if doc_type == 'распоряжение':
        match = re.search(r'\n(от[_\- ]+?)?\d\d\.\d\d\.\d\d\d\d[^\n]*\n*(№|ме) *([\w\-/а-я]+).?\n', doc)
        if match:
            return re.sub('_', '', match.group(3))
        
    if doc_type == 'приказ':
        match = re.search(r'приказ\n[^\n№]*\n*[^\n№]*№ ([\w\-/а-я ]+)', doc)
        if match:
            number = re.sub('_', '', match.group(1))
            number = re.sub(' -', '', number)
            return number.strip().split(' ')[-1]
    
    matches = [match[0] + match[1] for match in NUM_DATE_EXPR_2.findall(doc)]
    matches += [process_match((match[0], match[1], match[2])) + match[3]
               for match in NUM_DATE_EXPR_1.findall(doc)]
    for match in matches:
        if match.startswith(doc_date):
            number = match[10:]
            break
    
    if doc_type == 'закон':
        if number:
            number = re.sub(r'([^\-]*-[ко]з).*', r'\1', number)
    
    if not number and doc_type == 'приказ':
        matches = re.findall(r'(\d?\d [а-я]+ \d\d\d\d|\d\d\.\d\d\.\d\d\d\d)([^\n]+)\n', doc)
        if matches:
            number = re.sub('_', '', matches[0][1])
            number = re.sub(' -', '', number)
            return number.strip().split(' ')[-1]
            
    if not number and (doc_type == 'распоряжение' or doc_type == 'приказ'):
        match = re.search(r'распоряжение\n([^\n\d]*\n)?([^\n\d]*\n)?[^\n]+[ _]([\w\-/а-я]+)\n', doc)
        if match:
            return match.group(3)
        matches = re.findall(r'№ *([\w\-/а-я]+)', doc)
        number = matches[0] if matches else ""
            
    if (doc_type == 'закон' or doc_type == 'постановление') and not number:
            matches = re.findall(r'№ *([\w\-/а-я]+)', doc)
            number = matches[-1] if matches else ""
            
    if not number and (doc_type  == 'указ'):
        match = re.search(r'№[^\n\d]*([\w\-/а-я]+)\n', doc)
        if match:
            return match.group(1)
        
    return re.sub('_', '', number)

In [124]:
def predict(test, types, dates):
    results = []
    for doc, doc_type, doc_date in zip(test, types, dates):
        number = extract_number(doc, doc_type, doc_date)
        prediction = {"type": "",
                      "date": "",
                      "number": number,
                      "authority": "",
                      "name": ""}
        results.append(prediction)
    return results

In [125]:
from eval_module import quality

train = []
labeled = []
names = []
with open("train/gold_labels.txt", "r") as read_file:
    for doc_info in read_file.readlines():
        doc_dict = json.loads(doc_info)
        docname = 'train/txts/' + doc_dict['id'] + '.txt'
        names.append(docname)
        with open(docname, 'r') as f:
            train.append(f.read())
        labeled.append(doc_dict['label'])

true_types = np.array([x['type'] for x in labeled])
true_dates = np.array([x['date'] for x in labeled])
predicted = predict(train, true_types, true_dates)
quality(predicted, labeled)

{'date_accuracy': 0.0,
 'number_accuracy': 0.9719227877,
 'type_f1_score': 0.0,
 'name_jaccard': 0.0,
 'authority_jaccard': 0.0,
 'subtasks_improves': 1}

In [126]:
predicted_labels = np.array(predicted)
test_labels = np.array(labeled)
names = np.array(names)
for doc_type in ['федеральный закон', 'постановление', 'приказ', 'распоряжение','закон', 'указ']:
    ids = np.where(true_types == doc_type)
    print(f"{doc_type} ({ids[0].shape[0]}): {quality(predicted_labels[ids], test_labels[ids])['number_accuracy']}")

федеральный закон (192): 1.0
постановление (2459): 0.9674664498
приказ (77): 0.8181818182
распоряжение (430): 0.9930232558
закон (199): 0.959798995
указ (632): 0.9889240506


In [128]:
doc_type = 'приказ'
ids = np.where(true_types == doc_type)

y_test = [(x['number'], x['type'], x['date']) for x in test_labels[ids]]
y_pred = [x['number'] for x in predicted_labels[ids]]

wrong_docs = []
i = 0
for test, pred in zip(y_test, y_pred):
    if test[0].lower() != pred.lower():
        wrong_docs.append((names[ids][i], test, pred))
    i += 1
    
for doc_name, test, pred in wrong_docs:
    with open(doc_name, 'r') as f:
        print(f"right value = {test}")
        print(f"wrong value = {pred}")
        print(f"docname = {doc_name}")
        print(f.read())
        print("=====")

right value = ('61', 'приказ', '19.01.2015')
wrong value = предоставлении
docname = train/txts/73dd0ec6a668c0d8b7b60204d15e3364053dbcb3.txt
е 2. Признать утратившим силу приказ Министерства спорта Российской Федерации:
==== ОТ 12 февраля 2014 г. № 80 "Об утверждении формы соглашения о предоставлении -

 

ЗАРЕГИСТРИРОВАНО

 

| Регистрационный № 45020 |
от 27 евуаний ›

. : ю *

: ВАН РАНО На пКа да ееллубаае й

МИНИСТЕРСТВО СПОРТА РОССИЙСКОЙ ФЕДЕРАЦИИ
(МИНСПОРТ РОССИИ) | В
ПРИКАЗ 3

" 19 "ЯНВ&Ё‚Я 2015г_ `№ 61 | В
| | МОСКВА ' ` ВЕ

  

Об утверждении формы соглашения о предоставлении субсидий из федерального
бюджета бюджетам субъектов Российской Федерации на софинансирование
строительства объектов капитального строительства, находящихся в
государственной собственности субъектов Российской Федерации (муниципальной
собственности) .

В соответствии с пунктом 6 Правил о предоставлении субсидий из федерального
бюджета бюджетам субъектов Российской Федерации на софинансирование строительств